In [7]:
#import Pkg; Pkg.add("ImageIO")

using FileIO #this is needed for reading and writing files
using ImageIO
using Images 
using Interpolations #this is needed for interpolation
using Plots 
using Statistics 
using Clustering

function read_image(no_of_rock_units:: Int, W::Int, Nx::Int)

    #Load in the cleaned domain image
    
    #state the filename with a shorthand variable name for easier reference 
    pic_path = "M:/Dissertation Code/UistDomainCroppedFilled.png"
    domain_image = load(pic_path) #load image from named pic_path
    
    # Remove alpha channel from RGBA image
    domain_image_rgb = map(RGB, domain_image)  # convert RGBA to RGB


    #record the dimensions of the image 
    p, q = size(domain_image)
    #k = 3 #standard RGB channels 


    domain_img_rgb = channelview(domain_image_rgb)

    reshape_rgb_for_cluster = reshape(domain_img_rgb, (3, p*q)) #this reshapes the image into a 2D array with each pixel as a row and the rgb values as columns



    #cluster using k means to group similar RGB values together so that cluster IDs can be assigned in the model to each rock type
    no_clusters = no_of_rock_units #this is the number of different rock bodies / colours in the image including the sky and the fractures
    clustered_rgb = kmeans(reshape_rgb_for_cluster, no_clusters)
    cluster_ids = clustered_rgb.assignments #this is the cluster ID for each pixel in the image, used as a label later on
    cluster_centres = clustered_rgb.centers #this is the RGB value for each cluster centre; i.e. the mean colour in each cluster


    #sort the cluster ids based on cluster centroid vertical position 
    cluster_id_image = reshape(cluster_ids, (p, q)) #this reshapes the cluster ids back into a 2D array with the same dimensions as the original image

    #compute the spatial centroids

    cluster_xy = zeros(Float64, no_clusters, 2) #this will hold the x and y coordinates of the cluster centroids

    for i = 1:no_clusters
        clust_coords = findall(cluster_id_image .== i) #find the coordinates of the pixels in cluster number i 
        if !isempty(clust_coords) #ensure that the current cluster is not empty
            ys = getindex.(clust_coords, 1)
            xs = getindex.(clust_coords, 2)


            cluster_xy[i, 1] = mean(xs) #compute the mean x coordinate of the pixels in cluster i
            cluster_xy[i, 2] = mean(ys) #compute the mean y coordinate of the pixels in cluster i
        else
            cluster_xy[i,:] = Inf
        end
    end

    sort_cluster_ids = sortperm(cluster_xy[:, 2]) #sort the cluster ids based on the y coordinate of the cluster centroid

    unit_labels = zeros(Int, p, q) #this will hold the unit labels for each pixel in the image
    for i = 1:no_clusters
        unit_labels[cluster_id_image .== sort_cluster_ids[i]] .= i #assign the unit label to each pixel in the image based on the sorted cluster ids
    end

    cluster_id_image = reshape(unit_labels, p, q) #each pixel in this grid stores its cluster id kind of like a map of the original image but each pixel stored an ID instead of a colour


    #Define Model dimensions
    D = W * p / q #calculate model depth, this preserves the aspect ratio of the image 
    Nz = Int(floor(Nx * p/q))

    pix_width = W / p #calculate the pixel width in the model 
    pix_x_centres = pix_width/2 : pix_width: W - pix_width/2 #calculate the x coordinates of the pixel centres
    pix_z_centres = pix_width/2 : pix_width: D - pix_width/2 #calculate the z coordinates of the pixel centres

    image_grid_x = repeat(pix_x_centres', Nz, 1)# create a grid of x coordinates for the image
    image_grid_z = repeat(pix_z_centres, 1, Nx) # create a grid of z coordinates for the image

    #define model grid onto which we can interpolate 
    cell_width = W / Nx #calculate the cell width in the model

    #calculate the midpoints of grid cells 
    cell_x_centres = cell_width/2 : cell_width: W - cell_width/2 #calculate the x coordinates of the cell centres
    cell_z_centres = cell_width/2 : cell_width: D - cell_width/2 #calculate the z coordinates of the cell centres

    model_grid_x = repeat(cell_x_centres', Nz, 1) # create a grid of x coordinates for the model
    model_grid_z = repeat(cell_z_centres, 1, Nx) # create a grid of z coordinates for the model

    ###Interplation###
    #cluster_id_image_f = Float64.(cluster_id_image)
    #itp = interpolate(cluster_id_image_f, BSpline(Linear()), OnGrid())
    #interpolation = extrapolate(itp, Flat())
    

    #img_interp = zeros(Float64, length(cell_z_centres), length(cell_x_centres)) #initialise the matrix for the interpolated image 

    # Interpolation with bounds clamped
    #for i in 1:length(cell_z_centres)
    #    for j in 1:length(cell_x_centres)
   #         z_pos = clamp(cell_z_centres[i] / pix_width, 1.0, size(cluster_id_image, 1))
    #        x_pos = clamp(cell_x_centres[j] / pix_width, 1.0, size(cluster_id_image, 2))
    ##        img_interp[i, j] = interpolation[z_pos, x_pos]
    #    end
    #end

    ### Interpolation ###
    cluster_id_image_f = Float64.(cluster_id_image)
    
        # Define the real coordinates of the image pixel centers
    z_vals = range(0, stop=D, length=size(cluster_id_image, 1))  # depth (rows)
    x_vals = range(0, stop=W, length=size(cluster_id_image, 2))  # width (cols)

    # Create interpolator over real-world coordinates
    itp = interpolate((z_vals, x_vals), cluster_id_image_f, Gridded(Linear()))

    # Now evaluate directly at model grid centres (also in real-world coords)
    img_interp = zeros(Float64, Nz, Nx)

    for i in 1:Nz
        for j in 1:Nx
            z = cell_z_centres[i]  # real-world coordinate
            x = cell_x_centres[j]
            img_interp[i, j] = itp(z, x)  # safe, real-world interpolation
        end
    end

    
    heatmap(pix_x_centres, pix_z_centres, cluster_id_image; aspect_ratio=:equal, title="Original Image")
    heatmap(cell_x_centres, cell_z_centres, img_interp; aspect_ratio=:equal, title="Interpolated Units")

    return UInt8.(clamp.(round.(img_interp), 0, 255)), D, Nz #restrict the values to be between 0 and 255 for storage as an unsigned 8 bit integer 
    #return UInt8.(img_interp), D, Nz
end 

read_image (generic function with 1 method)

In [2]:
function diffusion(T::Array{Float64,2}, k0::Array{Float64, 2}, dx::Float64, ix::Vector{Int}, iz::Vector{Int}, geotherm::Tuple{Float64, Float64})
    #average thermal conductivity from the faces so that they 'lie' in the middle where the temp is
    kx = (k[:, ix[1:end-1]] + k[:, ix[2:end]]) ./ 2 #thermal conductivity in x direction
    kz = (k[iz[1:end-1], :] + k[:, iz[2:end]]) ./ 2 #thermal conductivity in z direction 

    

    qx= - kx .* diff(T[:, vec[ix]], dims=2) ./ dx #heat flux in x direction 
    qz = - kz .* diff(T[vec[iz], :], dims=1) ./ dx #heat flux in z direction

    qz[end, :] .= -kz[end, :] .* geotherm[2] #apply boundary condition at the bottom of the model

    dqx = diff(qx, dims=2) ./ dx #change in heat flux in x direction
    dqz = diff(qz, dims=1) ./ dx #change in heat flux in z direction

    dTdt_diffusion = - (dqx .+ dqz) #change in temperature due to diffusion

    return dTdt_diffusion
end



diffusion (generic function with 1 method)

In [8]:
#Define user specified parameters and image location 
using Statistics 

no_of_rock_units = 7
W = 3933
Nx = 100
D = 5000

tend = 1e6 #end time in years
k0 = 1e-6 #diffusion coefficient in m^2/s 

pic_path = "M:/Dissertation Code/UistDomainCroppedFilled.png"
domain_image = load(pic_path)
println("Image size: ", size(domain_image))
units, D, Nz = read_image(no_of_rock_units, W, Nx)
#read_image(width, no of cells sideways)


Image size: (799, 628)


┌ Warning: The clustering cost increased at iteration #3
└ @ Clustering C:\Users\2572664c\.julia\packages\Clustering\M6mjF\src\kmeans.jl:191
┌ Warning: The clustering cost increased at iteration #4
└ @ Clustering C:\Users\2572664c\.julia\packages\Clustering\M6mjF\src\kmeans.jl:191
┌ Warning: The clustering cost increased at iteration #5
└ @ Clustering C:\Users\2572664c\.julia\packages\Clustering\M6mjF\src\kmeans.jl:191


(UInt8[0x01 0x01 … 0x01 0x01; 0x01 0x01 … 0x01 0x01; … ; 0x07 0x07 … 0x07 0x07; 0x07 0x07 … 0x07 0x07], 5003.928343949045, 127)

In [ ]:
# ***** RUN 2D MODEL FROM IMAGE ***********************************

# Clear workspace equivalent
using LinearAlgebra
using Statistics

# Declare test and conservation flags

#const conservation = "NO"

# Load model setup from image, interpolate to target grid size
h = W / Nx        # grid spacing
n_units = 7       # number of rock units in image
img_interp, D, Nz = read_image(n_units, W, Nx)
# Sediment parameters
#rho_particle = 2650.0    # [kg/m³]
#rho_air = 1.225          # [kg/m³]
#sa_phi = 0.23
#gr_phi = 0.34
#si_phi = 0.15

#rho_sand = (1 - sa_phi) * rho_particle + sa_phi * rho_air
#rho_grav = (1 - gr_phi) * rho_particle + gr_phi * rho_air
#rho_silt = (1 - si_phi) * rho_particle + si_phi * rho_air

# Material properties: [kT, rho0, Cp, Hr]
matprop = [
 #   kT      rho0       Cp        Hr          # Material properties   
 1   1e-6    1           1000      0             # Sky 
 2   2.467   2700.0       874.5    2.9e-6       # Yellow: Mylonite 
 3   3.218   2703.5      1000      5.575e-6     # Red :Corodale Gneiss 
 4   0.25    1000        932      1e-6         # Orange: Crushged and Altered Corodale Gneiss 
 5   0.65    1000        566      1e-6         # Green: Corodale Gneiss
 6   1.3     2091.8       878      1e-6         # Purple: Pseudotachylite  
 7   0.39    1000        1088      1e-6         # Lilac: Basement Gneiss  

]


    
rho0 = reshape([matprop[u, 2] for u in n_units], Nz, Nx)
Cp   = reshape([matprop[u, 3] for u in n_units], Nz, Nx)
sigma = reshape([matprop[u, 1] for u in n_units], Nz, Nx)
Hr   = reshape([matprop[u, 4] for u in n_units], Nz, Nx)
air  = n_units .== 1
t_gradient = 35 / 1000
geotherm = [0.0, t_gradient]


a = rho0 .* Cp
kT0 = sigma .* 1000.0 ./ a

# Remaining model parameters
Ttop = 8.0
Tbot = Ttop + D * geotherm[2]
cT = 1e-9
mT = 2
g0 = 9.8
aT = 1e-4

yr = 60 * 60 * 24 * 365.25
endT = 1e6 * yr
CFL = 0.9
nop = 100

dTdto = 0.0

# Call appropriate model routine
#if conservation == "YES"
#    include("temperature_conservation_test.jl")
#    temperature_conservation_test()  
#else
#    include("helmsRK4.jl")
#    helmsRK4()  
#end


In [ ]:
ix = reshape(vcat(1, collect(1:Nx), Nx), 1, :)  #'create a padded vector with a cushion of 1 and Nx on each end 
iz = reshape(vcat(1, collect(1:Nz), Nz), :, 1) #do the same in the z direction  


Ttop = 11.74 #annual average temp from Met Office 
geotherm = [0, 30/100] #geothermal gradient in C/km typical values from 22-31 deg per km, ask about non linear geotherm 



cell_width = W / Nx #calculate the cell width in the model
cell_x_centres = cell_width/2 : cell_width: W - cell_width/2 #calculate the x coordinates of the cell centres
cell_z_centres = cell_width/2 : cell_width: D - cell_width/2 #calculate the z coordinates of the cell centres


T = Ttop .+ geotherm[2].*cell_z_centres #calculate the temperature at each depth based on the top temperature and the geothermal gradient
T_insant = T #store the current temperature for test plotting 
Tbottom = Ttop + geotherm[2] * D / 1000 #calculate the bottom temperature based on the top temperature and the geothermal gradient
Tair = Ttop 

rho_gradient = rho0 .* (1 .- aT .* (T .- Ttop)) #calculate the density gradient based on the temperature gradient and the thermal expansion coefficient

dt = CFL * ((cell_width/2)^2)/max(kT0) #calculate the time step based on the CFL condition and the thermal diffusivity 
println("Time step: ", dt)

t=0 
k=0 
dTdt = 0 

while t <= tend
    # Increment time and step count
    t += dt
    k += 1

    # Store old temperature and rate
    dTdto = dTdt
    To = T

    # Compute rate of change using Runge-Kutta 4th order method
    dTdt1 = diffusion(T, kT, h, ix3, iz3, geotherm)
    dTdt2 = diffusion(T .+ dTdt1 .* (dt/2), kT, h, ix3, iz3, geotherm)
    dTdt3 = diffusion(T .+ dTdt2 .* (dt/2), kT, h, ix3, iz3, geotherm)
    dTdt4 = diffusion(T .+ dTdt3 .* dt,    kT, h, ix3, iz3, geotherm)

    # Calculate heat source contribution 
    Hs = (Hr .* 1e-6) ./ (rho .* Cp)

    # Update temperature using the RK4 method 
    T .= T .+ (dTdt1 .+ 2 .* dTdt2 .+ 2 .* dTdt3 .+ dTdt4) .* (dt/6) .+ Hs

    # Keep air layer at constant temperature
    T[air] .= Ttop

    # Plot every 'nop' time steps
    if k % nop == 0
        makefig(xc, zc, T)
    end
end


